In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer
from skopt.space import Real, Integer
from skopt import BayesSearchCV
from sklearn.naive_bayes import GaussianNB

import numpy as np


In [ ]:
x_train_path = "../data/x_train.txt"
y_train_path = "../data/y_train.txt"

columns = [102, 103, 105]

x_data = np.loadtxt(x_train_path, delimiter=" ")
x_data = x_data[:, columns]
y_data = np.loadtxt(y_train_path, delimiter=" ")

print("X shape:", x_data.shape)
print("Y shape:", y_data.shape)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, test_size=0.2
    )


In [ ]:
def calc_money(y_true, y_pred, columns_indices: list[int]) -> float:
    num_correct = np.sum((y_true == 1) & (y_pred == 1))
    profit = (num_correct * 10) - len(columns_indices) * 200
    return profit


def custom_scorer(columns_indices):
    def scorer(y_true, y_pred):
        return calc_money(y_true, y_pred, columns_indices)
    return make_scorer(scorer)


In [ ]:
param_grid = {
    'n_estimators': Integer(10, 1000),        # Liczba drzew w lesie
    'max_features': Real(0.1, 0.999),         # Maksymalna liczba cech branych pod uwagę przy podziale
    'max_depth': Integer(1, 30),              # Maksymalna głębokość drzewa
    'min_samples_split': Integer(2, 10),      # Minimalna liczba próbek wymagana do podziału węzła
    'min_samples_leaf': Integer(1, 10),       # Minimalna liczba próbek wymagana w liściu
    'bootstrap': [True, False]                # Metoda bootstrap dla próbkowania
}

model = RandomForestClassifier()
scorer = make_scorer(calc_money)

opt = BayesSearchCV(
    estimator=model,
    search_spaces=param_grid,
    n_iter=32,                                     # Liczba iteracji poszukiwań
    cv=5,                                          # Liczba foldów w walidacji krzyżowej
    n_jobs=-1,                                     # Użycie wszystkich dostępnych rdzeni CPU
    random_state=42,                               # Ustalony seed dla powtarzalności wyników
    scoring=custom_scorer(columns_indices=columns) # 
)

_ = opt.fit(x_train, y_train)

print(opt.score(x_test, y_test))
